# Performance check

In [1]:
using LatticeQM
using Revise

In [2]:
lat = Geometries2D.honeycomb_twisted(30)
R1 = Structure.allpositions(lat)
R2 = R1
R2[1:2,:] .= R1[1:2,:] .+ (Structure.getA(lat) * [1; 0]);

Twist α=1.085°   (n,m)=(30,1)


In [3]:
using LinearAlgebra

function t(r::AbstractVector{Float64}; tz::Float64=0.46, t0::Float64=1.0, ℓinter::Float64=0.125, ℓintra::Float64=0.08, ℓz::Float64=0.01,z::Float64=3.0, a::Float64=1.0,
           Δmin::Float64=0.1, Δmax::Float64=5.0)
    @views Δ = sqrt(sum(abs2,r[1:3]))
    result = 0.0
    if Δmax > Δ > Δmin
        δz = r[3]
        χ = δz^2 /(Δ^2)
        result +=  t0 * χ * exp(-(Δ-z)/ℓinter) + tz * (1-χ) * exp(-(Δ-a)/ℓintra) * exp(-δz^2 /ℓz^2)
    end
    result
end

function t!(out::Vector{Float64}, r::Vector{Float64}; tz::Float64=0.46, t0::Float64=1.0, ℓinter::Float64=0.125, ℓintra::Float64=0.08, ℓz::Float64=0.01,z::Float64=3.0, a::Float64=1.0,
           Δmin::Float64=0.1, Δmax::Float64=5.0)
    @views out[1] = sqrt.(sum(abs2,r[1:3]))
    out[2] = 0.0
    if Δmax > out[1] > Δmin
        @views out[2] = r[3]^2 /(out[1]^2)
        @views out[3] =  t0 * out[2] * exp(-(out[1]-z)/ℓinter) + tz * (1-out[2]) * exp(-(out[1]-a)/ℓintra) * exp(-r[3]^2 /ℓz^2)
    end
    out
end

t! (generic function with 1 method)

In [44]:
const MAX_DENSE = 500
const MAX_DIAGS = 100

using SparseArrays
# using Base: gc_enable


function t(R1::Matrix{Float64}, R2::Matrix{Float64}; tmin=1e-7, tz::Float64=0.46, t0::Float64=1.0, ℓinter::Float64=0.125, ℓintra::Float64=0.08, ℓz::Float64=0.01,z::Float64=3.0, a::Float64=1.0,
           Δmin::Float64=0.1, Δmax::Float64=5.0,
           kwargs...)
    N = size(R1,2)

    # Preallocate memory: important for huge sparse matrices
    maxind = (N>MAX_DENSE) ? round(Int, MAX_DIAGS * N) : N^2 # MIN_SPARSITY * N^2 # semi-arbitrary limit for dense allocation
    IS = Vector{Int}(undef, maxind)
    JS = similar(IS)
    VS = similar(IS, Float64)
    
    δR = similar(R1)

    count = 1
    @time @fastmath @inbounds for j=1:N
        @views δR .= R1 .- R2[:,j]
        for i=1:N
            
            @views Δ = sqrt(sum(abs2,δR[1:3,i]))
            if Δmax < Δ || Δ < Δmin
                continue
            end
            
            δz = δR[3,i]
            χ = δz^2 /(Δ^2)
            v =  t0 * χ * exp(-(Δ-z)/ℓinter) + tz * (1-χ) * exp(-(Δ-a)/ℓintra) * exp(-δz^2 /ℓz^2)
            
            if abs(v) < tmin
                continue
            end
            
            IS[count], JS[count], VS[count] = i, j, v
            count = count+1
        end
    end
    count = count - 1

    @views sparse(IS[1:count],JS[1:count],complex(VS[1:count]), N, N)
end

t (generic function with 2 methods)

In [45]:
t(R1,R2)

  0.789774 seconds (11.16 k allocations: 523.313 KiB)


11164×11164 SparseMatrixCSC{Complex{Float64},Int64} with 517070 stored entries:
  [3    ,     1]  =  4.88301e-5+0.0im
  [1644 ,     1]  =  1.71426e-6+0.0im
  [5584 ,     1]  =  0.000188698+0.0im
  [5599 ,     1]  =  1.0+0.0im
  [5600 ,     1]  =  0.00545003+0.0im
  [5601 ,     1]  =  0.0183066+0.0im
  [5602 ,     1]  =  5.47916e-7+0.0im
  [5603 ,     1]  =  2.40765e-5+0.0im
  [5607 ,     1]  =  1.36017e-6+0.0im
  [5    ,     2]  =  1.71426e-6+0.0im
  [4107 ,     2]  =  1.71426e-6+0.0im
  [4108 ,     2]  =  4.88301e-5+0.0im
  ⋮
  [5575 , 11164]  =  2.6493e-5+0.0im
  [5576 , 11164]  =  0.000216478+0.0im
  [5577 , 11164]  =  2.17696e-5+0.0im
  [5578 , 11164]  =  0.00502988+0.0im
  [5579 , 11164]  =  0.0182579+0.0im
  [5580 , 11164]  =  0.256404+0.0im
  [5581 , 11164]  =  0.998449+0.0im
  [5582 , 11164]  =  0.256781+0.0im
  [5596 , 11164]  =  4.88301e-5+0.0im
  [5597 , 11164]  =  0.46+0.0im
  [11054, 11164]  =  0.46+0.0im
  [11161, 11164]  =  1.71426e-6+0.0im
  [11163, 11164]  =  4.88301e-

In [ ]:
# function t(R1::Matrix{Float64}, R2::Matrix{Float64}; tmin=1e-7,
#            kwargs...)
#     N = size(R1,2)

#     # Preallocate memory: important for huge sparse matrices
#     maxind = (N>MAX_DENSE) ? round(Int, MAX_DIAGS * N) : N^2 # MIN_SPARSITY * N^2 # semi-arbitrary limit for dense allocation
#     IS = Vector{Int}(undef, maxind)
#     JS = similar(IS)
#     VS = similar(IS, Float64)
    
#     δR = similar(R1)
# #     δr = similar(R1[:,1])
    
#     t0(args...) = t(args...; kwargs...)

#     count = 1
#     @time @fastmath @inbounds for j=1:N
#         @views δR .= R1 .- R2[:,j]
#         for i=1:N
#             @views v = t0(δR[:,i]) #1.007e-7 * rand() #1.4e-7 * rand() # t0(δr)
#             if abs(v) > tmin
#                 IS[count], JS[count], VS[count] = i, j, v
#                 count = count+1
#             end
#         end
#     end
#     count = count - 1

#     @views sparse(IS[1:count],JS[1:count],complex(VS[1:count]), N, N)
# end

In [ ]:
# function t(R1::Matrix{Float64}, R2::Matrix{Float64}; tmin=1e-7,
#            kwargs...)
#     N = size(R1,2)

#     # Preallocate memory: important for huge sparse matrices
#     maxind = (N>MAX_DENSE) ? round(Int, MAX_DIAGS * N) : N^2 # MIN_SPARSITY * N^2 # semi-arbitrary limit for dense allocation
#     IS = Vector{Int}(undef, maxind)
#     JS = similar(IS)
#     VS = similar(IS, Float64)
    
#     δr = similar(R1[:,1])
    
#     t0(args...) = t(args...; kwargs...)

#     count = 1
#     @time @fastmath @inbounds for (i,ri)=enumerate(eachcol(R1)), (j,rj)=enumerate(eachcol(R2))
#         δr .= ri.-rj
#         VS[count] = t0(δr) #1.4e-7 * rand() # t(δr)
#         if abs(VS[count]) > tmin
#             IS[count], JS[count] = i, j
#             count = count+1
#         end
#     end
#     count = count - 1
    

#     @views sparse(IS[1:count],JS[1:count],complex(VS[1:count]), N, N)
# end

In [ ]:
function t(R1::Matrix{Float64}, R2::Matrix{Float64}; tmin=1e-7, tz::Float64=0.46, t0::Float64=1.0, ℓinter::Float64=0.125, ℓintra::Float64=0.08, ℓz::Float64=0.01,z::Float64=3.0, a::Float64=1.0,
           Δmin::Float64=0.1, Δmax::Float64=5.0,
           kwargs...)
    N = size(R1,2)

    # Preallocate memory: important for huge sparse matrices
    maxind = (N>MAX_DENSE) ? round(Int, MAX_DIAGS * N) : N^2 # MIN_SPARSITY * N^2 # semi-arbitrary limit for dense allocation
    IS = Vector{Int}(undef, maxind)
    JS = similar(IS)
    VS = similar(IS, Float64)
    
    δr = similar(R1[:,1])
    
#     t0(args...) = t(args...; kwargs...)

    count = 1
    @time @fastmath @inbounds for (i,ri)=enumerate(eachcol(R1)), (j,rj)=enumerate(eachcol(R2))
        δr .= ri.-rj
        
        @views Δ = sqrt.(sum(abs2,δr[1:3]))
        VS[count] = 0.0
        if Δmax > Δ > Δmin
            χ = δr[3]^2 /(Δ^2)
            VS[count] +=  t0 * χ * exp(-(Δ-z)/ℓinter) + tz * (1-χ) * exp(-(Δ-a)/ℓintra) * exp(-δr[3]^2 /ℓz^2)
        end
        
#         VS[count] = t0(δr) #1.4e-7 * rand() # t(δr)
        if abs(VS[count]) > tmin
            IS[count], JS[count] = i, j
            count = count+1
        end
    end
    count = count - 1
    

    @views sparse(IS[1:count],JS[1:count],complex(VS[1:count]), N, N)
end